In [1]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
!pip install pmdarima
from pmdarima.arima import auto_arima
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf


     |████████████████████████████████| 1.5 MB 2.8 MB/s 
     |████████████████████████████████| 2.1 MB 22.8 MB/s 
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.21
    Uninstalling Cython-0.29.21:
      Successfully uninstalled Cython-0.29.21
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv') # train
submit = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv') # train

train['date'] = train['date'].astype('datetime64[ns]')

In [3]:
df = train[["item_cnt_day"]].copy()

In [4]:
df.item_cnt_day

0          1.0
1          1.0
2         -1.0
3          1.0
4          1.0
          ... 
2935844    1.0
2935845    1.0
2935846    1.0
2935847    1.0
2935848    1.0
Name: item_cnt_day, Length: 2935849, dtype: float64

In [5]:
model = auto_arima(df[0:155000], trace=True, error_action='ignore', suppress_warnings=True)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=332324.654, Time=216.21 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=408421.629, Time=25.62 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=371749.346, Time=13.84 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=338639.184, Time=39.09 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=408419.629, Time=2.40 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=319.14 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=332698.256, Time=142.54 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=332692.271, Time=232.71 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=371.35 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=334473.152, Time=70.92 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=305.64 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=332538.552, Time=215.47 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=433.64 sec
 ARIMA(2,1,2)(0,0,0)[0]             : AIC=inf, Time=76.04 sec

Best model:  ARIMA

In [6]:
result = model.fit(df[0:155000])

In [7]:
print(result.summary())

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:               155000
Model:               SARIMAX(2, 1, 2)   Log Likelihood             -166156.327
Date:                Thu, 31 Dec 2020   AIC                         332324.654
Time:                        10:05:18   BIC                         332384.361
Sample:                             0   HQIC                        332342.424
                             - 155000                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept    3.55e-06   1.79e-05      0.199      0.843   -3.15e-05    3.86e-05
ar.L1          0.7258      0.006    131.455      0.000       0.715       0.737
ar.L2         -0.0156      0.001    -10.642      0.0

In [8]:
pred = result.predict(214200)

In [9]:
pred = pd.Series(pred)

In [10]:
submit.item_cnt_month = pred

In [11]:
submit

,ID,item_cnt_month
0,0,1.079406
1,1,1.095416
2,2,1.105804
3,3,1.113099
4,4,1.118235
...,...,...
214195,214195,3.754721
214196,214196,3.754733
214197,214197,3.754745
214198,214198,3.754758


In [12]:
submit.to_csv('submission.csv', index=False)  